## Sprint 深層学習スクラッチ 畳み込みニューラルネットワーク2

In [1]:
import numpy as np
import random

In [2]:
from sklearn.metrics import accuracy_score

In [3]:
from keras.datasets import mnist
from sklearn.preprocessing import OneHotEncoder
(X_train, t_train), (X_test, t_test) = mnist.load_data()

/Users/tominagashuuji/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

In [5]:
enc = OneHotEncoder(handle_unknown="ignore", sparse=False)
t_train_one_hot = enc.fit_transform(t_train[:, np.newaxis])
t_test_one_hot = enc.fit_transform(t_test[:,  np.newaxis])

In [6]:
X_train = X_train.reshape(-1, 1, 28, 28)
X_test = X_test.reshape(-1, 1, 28, 28)

### 【問題1】2次元畳み込み層の作成¶
1次元畳み込み層のクラスConv1dを発展させ、2次元畳み込み層のクラスConv2dを作成してください。
フォワードプロパゲーションの数式は以下のようになります。

In [7]:
class Conv2d:
    
    def __init__(self, initializer, out_chanel, in_chanel, height, width, optimizer):
        init = initializer
        self.w = init.W(out_chanel, in_chanel, height, width)
        self.b = init.B(out_chanel)
        self.optimizer = optimizer
    
    def forward(self, X):
        self.sample_size, self.in_chanel, self.x_height, self.x_width = X.shape
        self.out_chanel, self.inchanel, self.w_height, self.w_width = self.w.shape
        self.XB = X
        
        
        A  = np.zeros([self.sample_size,  self.out_chanel, self.x_height -2, self.x_width -2])
        for n in range(self.sample_size):
            for outchan in range(self.out_chanel):
                for inchan in range(self.in_chanel):
                    for i in range(self.x_height-2):
                        for j in range(self.x_width-2):
                            sig = 0
                            for s in range(self.w_height):
                                for t in range(self.w_width):
                                    sig += X[n, inchan, i+s, j+t] * self.w[outchan, inchan, s, t]
                        A[n, outchan, i, j] += sig + self.b[outchan]
        return A
    
    def backward(self, dA):
        n_out_h , n_out_w = N_out(self.x_height, self.x_width, 0, self.w_height, self.w_width, 1)
        self.lb = dA.sum(axis=(0, 2, 3))
        
        self.lw = np.zeros_like(self.w)
        for n in range(self.sample_size):
            for m in range(self.out_chanel):
                for k in range(self.in_chanel):
                    for s in range(self.w_height):
                        for t in range(self.w_width):
                            for i in range(self.w_height-1):
                                for j in range(self.w_width-1):
                                    self.lw[m, k, s, t] += dA[n, m, i, j] * self.XB[n, k, i+s, j+t]
        
        
        dZ = np.zeros_like(self.XB)
        for n in range(self.sample_size):
            for m in range(self.out_chanel):
                for k in range(self.inchanel):
                    for i in range(self.x_height):
                        for j in range(self.x_width):
                            sig = 0
                            for s in range(self.w_height):
                                for t in range(self.w_width):
                                    if i-s<0 or i-s>n_out_h-1 or j-t < 0 or j-t>n_out_w-1:
                                        pass
                                    else:
                                        sig += dA[n, m, i-s, j-t] * self.w[m, k, s, t]
                            dZ[n, k, i, j] += sig
        
        
        
        self = self.optimizer.update(self)
        return dZ

### 【問題2】2次元畳み込み後の出力サイズ¶
畳み込みを行うと特徴マップのサイズが変化します。どのように変化するかは以下の数式から求められます。この計算を行う関数を作成してください。

In [8]:
def N_out(Nh_in, Nw_in, P, Fh, Fw, S):
    Nh_out = ((Nh_in  + 2*P - Fh) / S) + 1
    Nw_out = ((Nw_in + 2*P-Fw) / S) + 1
    return int(Nh_out), int(Nw_out)

### 【問題3】最大プーリング層の作成
最大プーリング層のクラスMaxPool2Dを作成してください。プーリング層は数式で表さない方が分かりやすい部分もありますが、数式で表すとフォワードプロパゲーションは以下のようになります。

In [9]:
class MaxPool2D:
    """
    pooling層のクラス
    2*2の正方行列内の最大値をリストに格納
    リストをreshapeすることで行列として返す
    
    2*2の正方行列内の最大値のインデックスを縦横それぞれ別のリストに格納
    要素が0の行列を作り、backwardで帰ってきた値を最大値のインデックスがあった場所へ代入
    """
    
    def forward(self,A):
        self.AB = A
        pooling_list = []
        self.h_list = []
        self.w_list = []
        for n in range(A.shape[0]):
            for cha in range(A.shape[1]):
                for h in range(0, 26, 2):
                    for w in range(0, 26, 2):
                        pooling_list = np.append(pooling_list, np.max(A[n, cha, h:h+2, w:w+2]))
                        self.h_list.append(np.unravel_index(np.argmax(A[n, cha, h:h+2, w:w+2]), A[n, cha, h:h+2, w:w+2].shape)[0])
                        self.w_list.append(np.unravel_index(np.argmax(A[n, cha, h:h+2, w:w+2]), A[n, cha, h:h+2, w:w+2].shape)[1])
                        
        pooing = pooling_list.reshape(A.shape[0], A.shape[1], 13, 13)
        return pooing
    
    def backward(self, dA):
        d = np.zeros_like(self.AB)
        i = 0
        for n in range(dA.shape[0]):
            for cha in range(dA.shape[1]):
                for h in range(0, 26, 2):
                    for w in range(0, 26, 2):
                        d[n, cha, h:h+2, w:w+2][self.h_list[i]][self.w_list[i]] = dA.reshape(-1)[i]
                        i+=1
        return d

### 【問題5】平滑化
平滑化するためのFlattenクラスを作成してください。

フォワードのときはチャンネル、高さ、幅の3次元を1次元にreshapeします。その値は記録しておき、バックワードのときに再びreshapeによって形を戻します。

この平滑化のクラスを挟むことで出力前の全結合層に適した配列を作ることができます。

In [10]:
class Flatten:
    def forward(self, X):
        self.X_shape = X.shape
        return X.reshape(X.shape[0], -1)
    
    def bakcward(self, dA):
        return dA.reshape(self.X_shape)

### 【問題6】学習と推定
作成したConv2dを使用してMNISTを学習・推定し、Accuracyを計算してください。

精度は低くともまずは動くことを目指してください。

#### クラス
#### FC

In [11]:
class FC:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        init = initializer
        self.n_nodes1 = n_nodes1
        self.w = init.W(n_nodes1, n_nodes2)
        self.b = init.B(n_nodes2)
    

    
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """
        self.z = X
        self.a = X@self.w + self.b
        
        return self.a

    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        dZ = dA @ self.w.T
        self.lw = self.z.T @ dA
        self.lb = np.sum(dA, axis=0)
        
        
        # 更新
        self = self.optimizer.update(self)
        return dZ

### FC initializer

In [12]:
class SimpleInitializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, sigma):
        self.sigma = sigma
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W :
        """
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        
        return W
    
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :
        """
        B  = self.sigma * np.random.randn(n_nodes2)
        return B

#### CNN Initializer

In [13]:
class SimpleInitializer_cnn:
    def __init__(self, sigma):
        self.sigma = sigma
        
    def W(self, out_chanel, in_chanel, height, width):

        W = self.sigma * np.random.randn(out_chanel, in_chanel, height, width)
        
        return W
    
    def B(self, out_chanel):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :
        """
        B  = self.sigma * np.random.randn(out_chanel)
        return B

### optimizer

In [14]:
class SGD:

    def __init__(self, lr):
        self.lr = lr
    def update(self, layer):
        
        layer.w = layer.w -  self.lr * layer.lw
        layer.b = layer.b - self.lr*layer.lb
        
        return layer

#### softmax

In [15]:
class Softmax:
    def forward(self, A):
        exp_a = np.exp(A)
        softmax_result = np.empty((A.shape[0], A.shape[1]))
        exp_sum = np.sum(exp_a, axis=1)
        for i in range(A.shape[0]):
            softmax_result[i] = exp_a[i] / exp_sum[i]
            
        return softmax_result
    
    def backward(self, Z, Y):
        
        L_A = Z - Y
        self.cross_entropy = -np.average(np.sum(Y*np.log(Z), axis=1))
        
        
        return L_A

#### Relu

In [16]:
class Relu:
    def forward(self, X):
        self.A = X
        return np.maximum(0, X)
    
    def backward(self, Z):
        
        return Z * np.maximum(np.sign(self.A), 0)

#### データ準備

In [17]:
X_train = X_train.reshape(-1, 1, 28, 28)
X_test = X_test.reshape(-1, 1, 28, 28)

In [18]:
#スクラッチが悪いせいで時間がかかるため、データ数を減らす
X_train_min = X_train[:3000]

In [19]:
t_train_min = t_train_one_hot[:3000]

In [20]:
X_test_min = X_test[:500]

In [21]:
t_test_min = t_test_one_hot[:500]

In [22]:
conv2d = Conv2d(SimpleInitializer_cnn(0.01), 3, 1, 3, 3, SGD(0.1))
maxpool = MaxPool2D()
relu = Relu()
fc1 = FC(507, 10, SimpleInitializer(0.01), SGD(0.1))
softmax = Softmax()

In [23]:
flat = Flatten()

#### 学習

In [24]:
A = conv2d.forward(X_train_min)

In [25]:
A_1 = relu.forward(A)

In [26]:
A_2 = maxpool.forward(A_1)

In [27]:
A_3 = flat.forward(A_2)

In [28]:
A_4 = fc1.forward(A_3)

In [29]:
A_5 = softmax.forward(A_4)

In [30]:
dA_1 = softmax.backward(A_5, t_train_min)

In [31]:
dA_2 = fc1.backward(dA_1)

In [32]:
dA_3 = flat.bakcward(dA_2)

In [33]:
dA_4 = maxpool.backward(dA_3)

In [34]:
dA_5 = relu.backward(dA_4)

In [35]:
dA_6 = conv2d.backward(dA_5)

### 推定

In [36]:
test_A = conv2d.forward(X_test_min)

In [37]:
test_A_2 = relu.forward(test_A)

In [38]:
test_A_3 = maxpool.forward(test_A_2)

In [39]:
test_A_4 = flat.forward(test_A_3)

In [40]:
test_A_5 = fc1.forward(test_A_4)

In [41]:
test_A_6 = softmax.forward(test_A_5)

In [42]:
y = np.argmax(test_A_6 , axis=1)

In [43]:
print(accuracy_score(t_test[:500], y))

0.134


### 【問題9】出力サイズとパラメータ数の計算
CNNモデルを構築する際には、全結合層に入力する段階で特徴量がいくつになっているかを事前に計算する必要があります。

また、巨大なモデルを扱うようになると、メモリや計算速度の関係でパラメータ数の計算は必須になってきます。フレームワークでは各層のパラメータ数を表示させることが可能ですが、意味を理解していなくては適切な調整が行えません。

以下の3つの畳み込み層の出力サイズとパラメータ数を計算してください。パラメータ数についてはバイアス項も考えてください。

1.
入力サイズ : 144×144, 3チャンネル  
フィルタサイズ : 3×3, 6チャンネル  
ストライド : 1  
パディング : なし  
出力サイズ：(6, 142, 142)  
パラメータ数:168  



2.
入力サイズ : 60×60, 24チャンネル  
フィルタサイズ : 3×3, 48チャンネル  
ストライド　: 1  
パディング : なし  
出力サイズ：(48, 58,58)  
パラメータ数:10416  



3.
入力サイズ : 20×20, 10チャンネル  
フィルタサイズ: 3×3, 20チャンネル  
ストライド : 2  
パディング : なし  
出力サイズ：(20, 9,9)  
パラメータ数:1820  